<a href="https://colab.research.google.com/github/rawanmt/CYS-507-Applied-Cryptography/blob/main/RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!pip install pycryptodome
from Crypto.Util import number
import random
from random import randint
import hashlib
import argparse

# Generating prime number
def generate_prime(size): 
  prime = number.getPrime(size)
  return prime

# Testing if a number is prime using Miller Rabin Test
def is_prime(p, r=10):
  for i in range(r):
    if not miller_rabin(p):
      return False
  return True

# Testing for primality , Miller Rabin Test
def miller_rabin(p):
    if p == 1: 
      return False
    if p == 2: 
      return True
    if p % 2 == 0: 
      return False

    m, k = p - 1, 0
    while m % 2 == 0:
        m, k = m // 2, k + 1
    a = randint(2, p - 1)
    x = pow(a, m, p)
    if x == 1 or x == p - 1: return True

    while k > 1:
        x = pow(x, 2, p)
        if x == 1: return False
        if x == p - 1: return True
        k = k - 1
    return False

# Testing if a number is prime using Miller Rabin Test
def is_prime(p, r=10):
  for i in range(r):
    if not miller_rabin(p):
      return False
  return True


# Calculate phi = (p - 1) * (q - 1)
def phi(num): 
    if(is_prime(num)):
        return num-1
    else:
        return False
 
# Caluclate e using Euclid's algorithm for finding the gcd
def generate_e(num): 
    def gcd(n1,n2):
        rest = 1
        while(n2 != 0):
            rest = n1%n2
            n1 = n2
            n2 = rest
        return n1

    while True:
        e = random.randrange(2,num) 
        if(gcd(num,e) == 1):
            return e

# Encrypt the message
def encryption(msg,e,n): # get the words and compute the cipher
    msg_size = len(msg)
    i = 0
    msg_list = []
    while(i < msg_size):
        letter = msg[i]
        k = ord(letter)
        k = k**e
        d = mod(k,n)
        msg_list.append(d)
        i += 1
    return msg_list


def mod(a,b): # mod function
    if(a<b):
        return a
    else:
        c=a%b
        return c   

# Calculate private key using Euclid's Extended algorithm to find the multiplicative inverse
def calculate_private_key(phi,e):
    d = 0
    while(mod(d*e,phi)!=1):
        d += 1
    return d


# Decrypt the message
def decryption(ciphrtext,n,d):
    msg_list = []
    i = 0
    msg_size = len(ciphrtext)
    while i < msg_size:
        result = ciphrtext[i]**d
        msg = mod(result,n)
        letters = chr(msg)
        msg_list.append(letters)
        i += 1
    return msg_list


def hasing(msg):
  hased_msg = hashlib.md5(msg.encode())
  return hased_msg.hexdigest()

def signature(msg,n,d):
  signed_msg = []
  for letters in msg:
    signature = (letters**d)%n 
    signed_msg.append(signature)
  return signed_msg

def verify_signiture(msg,n,e):
  verf_msg = []
  for letters in msg:
      verified = (letters**e)%n 
      verf_msg.append(verified)
  return verf_msg


## MAIN
if __name__=='__main__':
  print(  
 """
    |-------------------------------------------------------| 
                          ____  _____ ___ 
                         / __ \/ ___//   |
                        / /_/ /\__ \/ /| |
                       / _, _/___/ / ___ |
                      /_/ |_|/____/_/  |_|                  
      
      Encryption, Decryption, Digital Signiture, Verification  
    |--------------------------------------------------------|                                      
  """
  )

  print("\n|-------- Alice Sending Encrypted and Signed Message to Bob --------|\n")
  #  Generate two prime numbers p and q with size of either 1024, 2048, or 4096
  prime_size = int(input("Please Specify the Size of p and q to be Greater than or equal 10:"))

  if (prime_size >= 5):
    p = generate_prime(prime_size) # generates random P
    q = generate_prime(prime_size) # generates random Q

     # test if p and q are prime based on Miller Rabin Test
    if (is_prime(p) and is_prime(q)):
      n = p*q # compute N
      x = phi(p) # compute the phi of P
      y = phi(q) # compute the phi of Q
      phi_N = x*y # compute the phi of N
      e = generate_e(phi_N) # generate E
      d = calculate_private_key(phi_N,e) # generate d aka private key 
      public_key = (n, e) # generate public key

    #  asking the user to insert the message
      text = input("\nInsert the Message to be Encrypted: ")
    # removing white spaces from the text
      text= text.strip()

    # compute MD5 hash function
      print("\nMD5 Hash of The Message: ", hash(text),"\n")

    # compute encryption 
      text_cipher = encryption(text,e,n)
      print('The Encrypted Message:', text_cipher,"\n")

      # compute digital signiture 
      signed_message =  signature(text_cipher,n,d)
      print("The Signed Message: ", signed_message,"\n")

      # print public and private keys
      print('The Public Key:', public_key,"\n")
      print('The Private Key:', d ,"\n")

      print("\n|-------- Bob Receving Decrypted and Verified Message from Alice --------|\n")

      # print public and private keys
      print('The Public Key:', public_key,"\n")
      print('The Private Key:', d ,"\n")

      # compute decryption 
      original_text = decryption(text_cipher,n,d)
      print('The Decrypted Message:', *original_text)
      print(''.join(str(letter) for letter in original_text))


      # compute digital signiture  
      verified_message = verify_signiture(signed_message,n,e)
      print("The Verified Message: ",verified_message,"\n")

      print("\n|-------- Cryptographic Parameters --------|\n")
      print("p = ",p, "\n")
      print("q = ",q, "\n")
      print("N = ",n, "\n")
      print("e = ",e, "\n")
      print("d = ",d, "\n")
 
  else: 
    print("Size must be grater than or equal 10")



    |-------------------------------------------------------| 
                          ____  _____ ___ 
                         / __ \/ ___//   |
                        / /_/ /\__ \/ /| |
                       / _, _/___/ / ___ |
                      /_/ |_|/____/_/  |_|                  
      
      Encryption, Decryption, Digital Signiture, Verification  
    |--------------------------------------------------------|                                      
  

|-------- Alice Sending Encrypted and Signed Message to Bob --------|

Please Specify the Size of p and q to be Greater than or equal 10:10

Insert the Message to be Encrypted: hello world 

MD5 Hash of The Message:  1414498004727258030 

The Encrypted Message: [500973, 296675, 280377, 280377, 64756, 386548, 278623, 64756, 59176, 280377, 124745] 

The Signed Message:  [104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100] 

The Public Key: (502907, 363407) 

The Private Key: 264863 


|-------- Bob Receving Decrypted and 